<h2> In this notebook, we will look at our dataset to get a feeling about the dataset
</h2>

We have a dataset regarding songs, their genre and some features. The genre is the thing we are trying to predict

In [ ]:
import pandas as pd
import seaborn as sns
